In [2]:
pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bayesian-optimization

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lightgbm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-optimize==0.8.1

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scikit-learn==0.23.2

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pickle
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from utils import  *
from skopt import BayesSearchCV
%matplotlib inline
plt.style.use('fivethirtyeight')

## Part 2: Supervised Learning Model


## Load and clean the train and test dataset

In [8]:
mailout_train = pd.read_csv('../../data/mail_train.csv',sep=';')
mailout_test = pd.read_csv('../../data/mail_test.csv',sep=';')


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#process load data
attributes = pd.read_excel('../../data/attributes.xlsx', engine='openpyxl', skiprows = 1)

#train data
mailout_train = unknown_unify(mailout_train, attributes)
X = feature_transform(mailout_train)

#test data
mailout_test = unknown_unify(mailout_test, attributes)
X_sub = feature_transform(mailout_test)

#----!

y = X['RESPONSE']
X.drop(['RESPONSE'], axis = 1, inplace = True)

print(X.shape, X_sub.shape)

Creating PRAEGENDE_JUGENDJAHRE_DECADE feature
Creating PRAEGENDE_JUGENDJAHRE_MOVEMENT feature
Creating PRAEGENDE_JUGENDJAHRE_DECADE feature
Creating PRAEGENDE_JUGENDJAHRE_MOVEMENT feature
(42962, 364) (42833, 364)


## create prediction csv file

In [10]:
def predict_sub1(X_train, y_train, model, X_test, mailout_test,file_name):
    '''
    Trains the final model and exports Submission.xls file for uploading to the Kaggle competition
    
    Args: 
        X_train: DataFrame() - Train data 
        y_train: DataFrame() - Train data labels
        model: sklearn.model - Final model to train
        X_test: string - Test data to predict for the Kaggle competition.
    
    '''

    model.fit(X_train, y_train)

    y_sub = model.predict_proba(X_test)[:, 1]

    submission = pd.DataFrame({'LNR': mailout_test['LNR'], 'RESPONSE': y_sub})
    submission.to_csv(file_name, index=False)

## bayesian with simple tunning test

In [11]:
#Importing necessary libraries
import xgboost as xgb


In [12]:
from sklearn.model_selection import StratifiedKFold

# SETTINGS - CHANGE THESE TO GET SOMETHING MEANINGFUL
ITERATIONS = 10 # 1000
TRAINING_SIZE = 100000 # 20000000
TEST_SIZE = 25000



In [20]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx',
        subsample=0.8,
        eta=0.1
        
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'max_depth': (3.0, 10.0),
        'gamma': (1e-9, 1.0, 'log-uniform'),
        'n_estimators': (100.0, 120.0)
        
        
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [ ]:
# Fit the model
result = bayes_cv_tuner.fit(X.values, y.values, callback=status_print)

In [19]:
# Checking best parameters:
print(bayes_cv_tuner.best_params_)

# Checking best score:
print('Best ROC_AUC score: {:.2f}'.format(bayes_cv_tuner.best_score_))

OrderedDict([('gamma', 0.020491035004385), ('learning_rate', 0.022066991249460103), ('max_depth', 7), ('n_estimators', 116)])
Best ROC_AUC score: 0.76


In [23]:
parameters= dict([('gamma', 0.020491035004385), ('learning_rate', 0.022066991249460103), ('max_depth', 7), ('n_estimators', 116)])

In [25]:
num_features = 366

#get the feature importances DataFrame and plot the barchart
model_fi = XGBClassifier( verbose = False).fit( X, y)
fi = pd.DataFrame({'features':X.columns, 'importance' : model_fi.feature_importances_/ model_fi.feature_importances_.sum()}).sort_values(['importance'], ascending = False)


#reduce the ftrain and test data based on the important features
X_reduced = X[fi['features'].head(num_features).to_list()]
X_sub_reduced = X_sub[fi['features'].head(num_features).to_list()]

[09:38:00] WARNING: ../src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:38:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [26]:
#use the dictionaty with optimized parameters result of the BayesianOptimization
model = XGBClassifier(**parameters)

#predefined funciton for trainings and exporting the final model
predict_sub(X_reduced, y, model, X_sub_reduced,mailout_test)


[09:38:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,10),
    'subsample': (1.0, 1.0),  # Change for big datasets
    'colsample': (1.0, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)

    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma)
    return np.mean(cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc'))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

In [ ]:
optimizer.maximize(n_iter=5, init_points=8, acq='ei')